# Interactive SOFC Microstructure Exploration

This notebook provides an interactive exploration of the 3D SOFC microstructure data, allowing you to:
- Generate microstructures with different parameters
- Visualize 3D data interactively
- Analyze phase distributions and properties
- Generate computational meshes
- Explore interface characteristics

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import h5py
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

from microstructure_generator import SOFCMicrostructureGenerator
from microstructure_analysis import MicrostructureAnalyzer
from mesh_generator import MeshGenerator
from visualization_dashboard import MicrostructureVisualizer

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("Setup complete! Ready for interactive exploration.")

## 1. Interactive Microstructure Generation

Use the widgets below to adjust parameters and generate different microstructures.

In [ ]:
# Create interactive widgets for parameter adjustment
resolution_widget = widgets.IntSlider(
    value=128,
    min=64,
    max=512,
    step=64,
    description='Resolution:',
    style={'description_width': 'initial'}
)

voxel_size_widget = widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.05,
    description='Voxel Size (μm):',
    style={'description_width': 'initial'}
)

porosity_widget = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.5,
    step=0.05,
    description='Porosity:',
    style={'description_width': 'initial'}
)

ni_ysz_ratio_widget = widgets.FloatSlider(
    value=0.6,
    min=0.3,
    max=0.8,
    step=0.1,
    description='Ni/YSZ Ratio:',
    style={'description_width': 'initial'}
)

ysz_thickness_widget = widgets.FloatSlider(
    value=10.0,
    min=5.0,
    max=20.0,
    step=1.0,
    description='YSZ Thickness (μm):',
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='Generate Microstructure',
    button_style='success',
    icon='play'
)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>Microstructure Parameters</h3>"),
    resolution_widget,
    voxel_size_widget,
    porosity_widget,
    ni_ysz_ratio_widget,
    ysz_thickness_widget,
    generate_button
]))

In [ ]:
# Global variables to store generated data
current_microstructure = None
current_voxel_size = None
current_generator = None

def generate_microstructure(b):
    """Generate microstructure with current parameters."""
    global current_microstructure, current_voxel_size, current_generator
    
    print("Generating microstructure...")
    
    # Get current parameter values
    resolution = (resolution_widget.value, resolution_widget.value, resolution_widget.value // 2)
    voxel_size = voxel_size_widget.value
    porosity = porosity_widget.value
    ni_ysz_ratio = ni_ysz_ratio_widget.value
    ysz_thickness = ysz_thickness_widget.value
    
    # Create generator
    current_generator = SOFCMicrostructureGenerator(
        resolution=resolution,
        voxel_size=voxel_size,
        porosity=porosity,
        ni_ysz_ratio=ni_ysz_ratio,
        ysz_thickness=ysz_thickness
    )
    
    # Generate microstructure
    current_microstructure = current_generator.generate_microstructure()
    current_voxel_size = voxel_size
    
    print(f"Generated microstructure with shape {current_microstructure.shape}")
    print("Phase distribution:")
    for phase, props in current_generator.phase_properties['phases'].items():
        print(f"  {phase}: {props['volume_fraction']:.3f}")
    
    # Update visualization
    update_visualization()

# Connect button to function
generate_button.on_click(generate_microstructure)

print("Click 'Generate Microstructure' to create your first microstructure!")

## 2. Interactive 3D Visualization

Visualize the generated microstructure in 3D with interactive controls.

In [ ]:
# Visualization controls
max_voxels_widget = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step=1000,
    description='Max Voxels:',
    style={'description_width': 'initial'}
)

opacity_widget = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Opacity:',
    style={'description_width': 'initial'}
)

display(widgets.HBox([max_voxels_widget, opacity_widget]))

In [ ]:
def update_visualization():
    """Update 3D visualization with current parameters."""
    if current_microstructure is None:
        print("No microstructure generated yet. Please generate one first.")
        return
    
    # Create visualizer
    visualizer = MicrostructureVisualizer(current_microstructure, current_voxel_size)
    
    # Create 3D plot
    fig = visualizer.create_3d_plotly_visualization(
        max_voxels=max_voxels_widget.value,
        opacity=opacity_widget.value
    )
    
    # Display plot
    fig.show()

# Connect widgets to update function
max_voxels_widget.observe(lambda change: update_visualization(), names='value')
opacity_widget.observe(lambda change: update_visualization(), names='value')

print("Visualization controls connected. Adjust sliders to update the 3D plot.")

## 3. Cross-Sectional Analysis

Explore cross-sections of the microstructure at different z-positions.

In [ ]:
def plot_cross_sections():
    """Plot cross-sections of the microstructure."""
    if current_microstructure is None:
        print("No microstructure generated yet.")
        return
    
    # Create visualizer
    visualizer = MicrostructureVisualizer(current_microstructure, current_voxel_size)
    
    # Create cross-section plot
    fig = visualizer.create_cross_section_plots()
    fig.show()

# Create button for cross-sections
cross_section_button = widgets.Button(
    description='Show Cross-Sections',
    button_style='info',
    icon='image'
)
cross_section_button.on_click(lambda b: plot_cross_sections())

display(cross_section_button)

## 4. Phase Distribution Analysis

Analyze the distribution and properties of different phases.

In [ ]:
def analyze_phases():
    """Analyze phase distribution and properties."""
    if current_microstructure is None:
        print("No microstructure generated yet.")
        return
    
    # Create visualizer
    visualizer = MicrostructureVisualizer(current_microstructure, current_voxel_size)
    
    # Create phase distribution plot
    fig = visualizer.create_phase_distribution_plots()
    fig.show()
    
    # Print detailed analysis
    print("\nDetailed Phase Analysis:")
    print("-" * 40)
    
    total_voxels = np.prod(current_microstructure.shape)
    phase_names = {0: 'Pore', 1: 'Ni', 2: 'YSZ_Anode', 3: 'YSZ_Electrolyte', 4: 'Interlayer'}
    
    for phase_id, phase_name in phase_names.items():
        count = np.sum(current_microstructure == phase_id)
        volume_fraction = count / total_voxels
        volume_um3 = count * (current_voxel_size ** 3)
        
        print(f"{phase_name}:")
        print(f"  Count: {count:,} voxels")
        print(f"  Volume fraction: {volume_fraction:.3f}")
        print(f"  Volume: {volume_um3:.2f} μm³")
        print()

# Create button for phase analysis
phase_analysis_button = widgets.Button(
    description='Analyze Phases',
    button_style='warning',
    icon='chart-bar'
)
phase_analysis_button.on_click(lambda b: analyze_phases())

display(phase_analysis_button)

## 5. Advanced Analysis

Perform advanced analysis including connectivity, pore network, and interface properties.

In [ ]:
def run_advanced_analysis():
    """Run comprehensive advanced analysis."""
    if current_microstructure is None:
        print("No microstructure generated yet.")
        return
    
    print("Running advanced analysis...")
    
    # Create analyzer
    analyzer = MicrostructureAnalyzer(current_microstructure, current_voxel_size)
    
    # Run analyses
    print("  Analyzing phase connectivity...")
    connectivity = analyzer.analyze_phase_connectivity()
    
    print("  Analyzing pore network...")
    pore_network = analyzer.analyze_pore_network()
    
    print("  Analyzing interface properties...")
    interfaces = analyzer.analyze_interface_properties()
    
    print("  Estimating mechanical properties...")
    mechanical = analyzer.estimate_mechanical_properties()
    
    # Display results
    print("\nAdvanced Analysis Results:")
    print("=" * 50)
    
    print("\nPhase Connectivity:")
    for phase, props in connectivity.items():
        print(f"  {phase}: {props['n_components']} components, {props['connectivity_percentage']:.1f}% connected")
    
    print("\nPore Network:")
    print(f"  Tortuosity: {pore_network['tortuosity']:.3f}")
    print(f"  Connectivity: {pore_network['connectivity']:.1f}%")
    print(f"  Mean pore size: {pore_network['mean_pore_size']:.2f} μm³")
    print(f"  Number of pores: {pore_network['n_pores']}")
    
    print("\nInterfaces:")
    for interface, props in interfaces.items():
        print(f"  {interface}: {props['area_um2']:.2f} μm²")
        if 'roughness_um' in props:
            print(f"    Roughness: {props['roughness_um']:.3f} μm")
    
    print("\nMechanical Properties:")
    print(f"  Effective Young's modulus: {mechanical['effective_youngs_modulus_Pa']/1e9:.1f} GPa")
    print(f"  Effective Poisson's ratio: {mechanical['effective_poisson_ratio']:.3f}")
    print(f"  Effective density: {mechanical['effective_density_kg_m3']:.0f} kg/m³")
    print(f"  Porosity: {mechanical['porosity']:.3f}")
    print(f"  Relative density: {mechanical['relative_density']:.3f}")

# Create button for advanced analysis
advanced_analysis_button = widgets.Button(
    description='Run Advanced Analysis',
    button_style='danger',
    icon='cogs'
)
advanced_analysis_button.on_click(lambda b: run_advanced_analysis())

display(advanced_analysis_button)

## 6. Mesh Generation

Generate computational meshes for finite element analysis.

In [ ]:
def generate_meshes():
    """Generate computational meshes."""
    if current_microstructure is None:
        print("No microstructure generated yet.")
        return
    
    print("Generating computational meshes...")
    
    # Create mesh generator
    mesh_gen = MeshGenerator(current_microstructure, current_voxel_size)
    
    # Generate different mesh types
    print("  Generating structured hexahedral mesh...")
    hex_mesh = mesh_gen.generate_structured_hex_mesh()
    
    print("  Generating unstructured tetrahedral mesh...")
    tet_mesh = mesh_gen.generate_unstructured_tet_mesh()
    
    print("  Generating surface mesh...")
    surface_mesh = mesh_gen.generate_surface_mesh()
    
    print("  Generating interface mesh...")
    interface_mesh = mesh_gen.generate_interface_mesh()
    
    # Calculate statistics
    hex_stats = mesh_gen.create_mesh_statistics(hex_mesh)
    tet_stats = mesh_gen.create_mesh_statistics(tet_mesh) if tet_mesh.n_cells > 0 else {}
    
    # Display results
    print("\nMesh Generation Results:")
    print("=" * 40)
    print(f"Hexahedral mesh: {hex_stats['n_points']:,} points, {hex_stats['n_cells']:,} cells")
    if tet_stats:
        print(f"Tetrahedral mesh: {tet_stats['n_points']:,} points, {tet_stats['n_cells']:,} cells")
    
    if 'mean_element_quality' in hex_stats:
        print(f"Mean element quality: {hex_stats['mean_element_quality']:.3f}")
    
    print("\nMeshes generated successfully!")
    print("You can export these meshes to VTK, STL, or other formats for use in FEA software.")

# Create button for mesh generation
mesh_generation_button = widgets.Button(
    description='Generate Meshes',
    button_style='success',
    icon='cube'
)
mesh_generation_button.on_click(lambda b: generate_meshes())

display(mesh_generation_button)

## 7. Save Results

Save the generated microstructure and analysis results.

In [ ]:
def save_results():
    """Save microstructure and analysis results."""
    if current_microstructure is None:
        print("No microstructure generated yet.")
        return
    
    print("Saving results...")
    
    # Create output directory
    output_dir = Path("notebook_output")
    output_dir.mkdir(exist_ok=True)
    
    # Save microstructure
    current_generator.save_hdf5(str(output_dir / "microstructure.h5"))
    current_generator.save_vtk(str(output_dir / "microstructure.vtk"))
    
    # Run and save analysis
    analyzer = MicrostructureAnalyzer(current_microstructure, current_voxel_size)
    report = analyzer.create_comprehensive_report()
    report.to_csv(str(output_dir / "analysis_report.csv"))
    
    # Save visualizations
    visualizer = MicrostructureVisualizer(current_microstructure, current_voxel_size)
    visualizer.save_static_plots(str(output_dir / "plots"))
    
    print(f"Results saved to {output_dir}/")
    print("Files created:")
    print("  - microstructure.h5 (HDF5 format)")
    print("  - microstructure.vtk (VTK format)")
    print("  - analysis_report.csv (Analysis results)")
    print("  - plots/ (Visualization files)")

# Create button for saving results
save_button = widgets.Button(
    description='Save Results',
    button_style='primary',
    icon='save'
)
save_button.on_click(lambda b: save_results())

display(save_button)

## 8. Summary and Next Steps

This notebook provides a complete interactive workflow for SOFC microstructure generation and analysis. You can:

1. **Generate microstructures** with different parameters
2. **Visualize 3D data** interactively
3. **Analyze phase distributions** and properties
4. **Perform advanced analysis** including connectivity and mechanical properties
5. **Generate computational meshes** for FEA
6. **Save results** for further analysis

### Next Steps:
- Export meshes to FEA software (ANSYS, Abaqus, etc.)
- Use the generated data for computational modeling
- Customize parameters for specific applications
- Explore different microstructure configurations

### Tips:
- Start with smaller resolutions for faster generation
- Use the visualization controls to explore different views
- Save interesting configurations for later use
- Experiment with different parameter combinations